# **Análisis de texto de eventos históricos**

## **Declaración del conjunto de datos**

Contamos con un dataset en formato JSON proveniente del repositorio 'awesome-json-datasets' en la sección 'Historical Events' sobre eventos históricos (disponible en: https://github.com/jdorfman/awesome-json-datasets). Este dataset cuenta con información desde el año 299 A.C. hasta el año 2013. Recopila sucesos importantes en el mundo a lo largo de este periodo señalado.

La estrucutra de cada recopilación es la siguiente:

```
{
    "date": "fecha del acontecimiento",
    "description": "descripción del evento en cuestión",
    "lang": "lenguaje de la descripción",
    "category1": "catergoría interna del dataset",
    "granularity": "granularidad"
}
```
Como se puede ver, no cuenta con una estructura compleja, y sus campos más importantes son 'date' que nos indica la fecha del suceso y 'description' donde se encuentran todos los detalles del evento. Este dataset cuenta con 20.330 registros diferentes.


## **Planteamiento de la problemática y diseño de la solución (tecnologías a implementar)**

Se plantea realizar un análisis descriptivo de esta información a nivel de país, agrupando sus eventos históricos y ver qué palabras son recurrentes en estos eventos. Así nos podemos dar una rápida percepción de la historia de un país en concreto. También se plantea analizar palabras clave en los eventos históricos como lo son 'guerra', 'atentados', 'ataque', 'muertos', 'descubrimiento', 'invención' y ver que tan concurrentes son a lo largo de la historia.


Para esta labor, nos apoyaremos de la herramienta MongoDB en su entorno de Python Pymongo. Este sistema de base de datos NoSQL nos ayudará a manejar adecuadamente el formato de este dataset (JSON) y más importante aún con el tratamiento de textos. Para esto último nos apoyaremos en dos funcionalidades de MongoDB: En el uso de expresiones regulares para busqueda en campos de texto y en las operaciones Map-Reduce. Junto con MongoDB, nos apoyaremos en las librerías propias de analítica de datos de Python. Con esto se pretenderá alcanzar los objetivos de este proyecto.

## **Descripción de la implementación realizada**

### **Instalación de herramientas**

Vamos a instalar _MongoDB_ en la máquina del entorno de ejecución con los comandos **`apt update`** y **`apt install`**:

In [1]:
# Actualizamos el repositorio e instalamos mongodb.
!sudo apt -qq update
!sudo apt -qq install -y mongodb

79 packages can be upgraded. Run 'apt list --upgradable' to see them.
mongodb is already the newest version (1:3.6.3-0ubuntu1.4).
0 upgraded, 0 newly installed, 0 to remove and 79 not upgraded.


In [2]:
# Ejecute esta celda para instalar la versión más reciente
!pip install -U plotly
!pip install -U kaleido # Necesitamos esta librería para exportar las visualizaciones como imágenes estáticas

Creamos las carpetas **`data`** y  **`log`** y ejecutamos el servicio **`mongod`**, tal como se realizó en la guía de operaciones en consola:

In [3]:
# Creamos los directorios 'data' y 'log'.
!mkdir -p data log

# Ejecutamos el servicio de Mongo.
!mongod --dbpath ./data --logpath ./log/mongod.log --fork

about to fork child process, waiting until server is ready for connections.
forked process: 3382
ERROR: child process failed, exited with error number 100
To see additional information in this output, start without the "--fork" option.


In [4]:
import pymongo
import numpy as np
import pandas as pd
import matplotlib as mpl
import datetime
from datetime import datetime
import re
import plotly
import plotly.graph_objs as go 
import plotly.express as px
import bson # Módulo bson (Binary JSON)
from ipywidgets import interact
import csv
from IPython.display import display, Markdown, Latex

### Adquisición e integración de datos

In [5]:
!wget -q --no-check-certificate 'https://drive.google.com/uc?id=1HrwF-wdeiM-9jMLiW2w68B5zjD2necOD&export=download' -O eventos_historicos.json

In [6]:
!wget -q --no-check-certificate 'https://drive.google.com/uc?id=1HrodNbzbE7zZrshRfNF3E816ECPcyho3&export=download' -O paises.csv

### Conexión al servidor de MongoDB

In [7]:
client = pymongo.MongoClient("localhost", 27017)
client

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

In [8]:
!mongoimport --db proyecto --collection eventos --drop --file eventos_historicos.json

2021-12-18T19:10:07.206+0000	connected to: localhost
2021-12-18T19:10:07.206+0000	dropping: proyecto.eventos
2021-12-18T19:10:07.769+0000	imported 20330 documents


In [9]:
db = client.proyecto
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'proyecto')

In [10]:
df = pd.DataFrame(db.eventos.find())
df

,_id,date,description,lang,granularity,category1,category2
0,61be320fc26e4275b500a6e7,-299,"Los Samnitas, inician los preparativos de la T...",es,year,NaN,NaN
1,61be320fc26e4275b500a6e8,-298,Inicio de la Tercera Guerra Samnita entre la R...,es,year,NaN,NaN
2,61be320fc26e4275b500a6e9,-297,"Roma conquista Rodas, anexionándola al Imperio...",es,year,NaN,NaN
3,61be320fc26e4275b500a6ea,-297,Bitinia declara formalmente su independencia c...,es,year,NaN,NaN
4,61be320fc26e4275b500a6eb,-297,"Batalla de Tiferno, entre Roma y el pueblo itá...",es,year,NaN,NaN
...,...,...,...,...,...,...,...
20325,61be320fc26e4275b500f660,2013/05/10,Posible lanzamiento del telescopio espacial Ja...,es,year,Junio,NaN
20326,61be320fc26e4275b500f661,2013/08/10,Inicio de nueva legislatura en Ecuador.,es,year,Agosto,NaN
20327,61be320fc26e4275b500f662,2013/08/15,"Toma de posesión del candidato electo, en las ...",es,year,Agosto,NaN
20328,61be320fc26e4275b500f663,2013/11/10,Elecciones presidenciales en Honduras.,es,year,Noviembre,NaN


In [11]:
db.eventos.find_one()

{'_id': ObjectId('61be320fc26e4275b500a6e7'),
 'date': '-299',
 'description': 'Los Samnitas, inician los preparativos de la Tercera guerra Samnita contra los romanos, reclutando tropas mercenarias de Galos y Sabinos y estableciendo alianzas con Etruscos y otros pueblos itálicos.',
 'granularity': 'year',
 'lang': 'es'}

In [12]:
db.eventos.create_index([('description', 'text')])

'description_text'

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20330 entries, 0 to 20329
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   _id          20330 non-null  object
 1   date         20330 non-null  object
 2   description  20330 non-null  object
 3   lang         20330 non-null  object
 4   granularity  20330 non-null  object
 5   category1    10796 non-null  object
 6   category2    313 non-null    object
dtypes: object(7)
memory usage: 1.1+ MB


### Preprocesamiento de los datos

In [14]:
df['date'].unique()

array(['-299', '-298', '-297', ..., '2013/08/15', '2013/11/10',
       '2013/11/17'], dtype=object)

No fue posible pasar todas las fechas a tipo de dato Date en Mongo, debido a que la conversión de fechas de pymongo se hace por medio de la librería datatime, y esta no permite años inferiores a 1000 (por ende mucho menos los años antes de Cristo), por tanto se decidió dejar como String

In [15]:
for date in df['date'].unique():
  if '/' not in date:
    new_date = date+'/01/01'
    db.eventos.update_many({'date' : date}, {'$set' : {"date" : new_date}}) 

In [16]:
df = pd.DataFrame(db.eventos.find())
df

,_id,date,description,lang,granularity,category1,category2
0,61be320fc26e4275b500a6e7,-299/01/01,"Los Samnitas, inician los preparativos de la T...",es,year,NaN,NaN
1,61be320fc26e4275b500a6e8,-298/01/01,Inicio de la Tercera Guerra Samnita entre la R...,es,year,NaN,NaN
2,61be320fc26e4275b500a6e9,-297/01/01,"Roma conquista Rodas, anexionándola al Imperio...",es,year,NaN,NaN
3,61be320fc26e4275b500a6ea,-297/01/01,Bitinia declara formalmente su independencia c...,es,year,NaN,NaN
4,61be320fc26e4275b500a6eb,-297/01/01,"Batalla de Tiferno, entre Roma y el pueblo itá...",es,year,NaN,NaN
...,...,...,...,...,...,...,...
20325,61be320fc26e4275b500f660,2013/05/10,Posible lanzamiento del telescopio espacial Ja...,es,year,Junio,NaN
20326,61be320fc26e4275b500f661,2013/08/10,Inicio de nueva legislatura en Ecuador.,es,year,Agosto,NaN
20327,61be320fc26e4275b500f662,2013/08/15,"Toma de posesión del candidato electo, en las ...",es,year,Agosto,NaN
20328,61be320fc26e4275b500f663,2013/11/10,Elecciones presidenciales en Honduras.,es,year,Noviembre,NaN


In [17]:
df['date'].unique()

array(['-299/01/01', '-298/01/01', '-297/01/01', ..., '2013/08/15',
       '2013/11/10', '2013/11/17'], dtype=object)

Creamos una copia de la colección para hacer un tratamiento especial - retirar las stop words

In [18]:
pipeline = [ {"$match": {}}, 
             {"$out": "eventos_stopwords"},
]
db.eventos.aggregate(pipeline)

In [19]:
stopwords = ["algún","alguna","algunas","alguno","algunos","ambos","ampleamos","ante","antes","aquel","aquellas","aquellos","aqui","arriba","atras","bajo","bastante","bien","cada","cierta","ciertas","cierto","ciertos","como","con","conseguimos","conseguir","consigo","consigue","consiguen","consigues","cual","cuando","dentro","desde","donde","dos","el","ellas","ellos","empleais","emplean","emplear","empleas","empleo","en","encima","entonces","entre","era","eramos","eran","eras","eres","es","esta","estaba","estado","estais","estamos","estan","estoy","fin","fue","fueron","fui","fuimos","gueno","ha","hace","haceis","hacemos","hacen","hacer","haces","hago","incluso","intenta","intentais","intentamos","intentan","intentar","intentas","intento","ir","la","largo","las","lo","los","mientras","mio","modo","muchos","muy","nos","nosotros","otro","para","pero","podeis","podemos","poder","podria","podriais","podriamos","podrian","podrias","por","por qué","porque","primero","puede","pueden","puedo","quien","sabe","sabeis","sabemos","saben","saber","sabes","ser","si","siendo","sin","sobre","sois","solamente","solo","somos","soy","su","sus","también","teneis","tenemos","tener","tengo","tiempo","tiene","tienen","todo","trabaja","trabajais","trabajamos","trabajan","trabajar","trabajas","trabajo","tras","tuyo","ultimo","un","una","unas","uno","unos","usa","usais","usamos","usan","usar","usas","uso","va","vais","valor","vamos","van","vaya","verdad","verdadera","verdadero","vosotras","vosotros","voy","yo","él","ésta","éstas","éste","éstos","última","últimas","último","últimos","a","añadió","aún","actualmente","adelante","además","afirmó","agregó","ahí","ahora","al","algo","alrededor","anterior","apenas","aproximadamente","aquí","así","aseguró","aunque","ayer","buen","buena","buenas","bueno","buenos","cómo","casi","cerca","cinco","comentó","conocer","consideró","considera","contra","cosas","creo","cuales","cualquier","cuanto","cuatro","cuenta","da","dado","dan","dar","de","debe","deben","debido","decir","dejó","del","demás","después","dice","dicen","dicho","dieron","diferente","diferentes","dijeron","dijo","dio","durante","e","ejemplo","ella","ello","embargo","encuentra","esa","esas","ese","eso","esos","está","están","estaban","estar","estará","estas","este","esto","estos","estuvo","ex","existe","existen","explicó","expresó","fuera","gran","grandes","había","habían","haber","habrá","hacerlo","hacia","haciendo","han","hasta","hay","haya","he","hecho","hemos","hicieron","hizo","hoy","hubo","igual","indicó","informó","junto","lado","le","les","llegó","lleva","llevar","luego","lugar","más","manera","manifestó","mayor","me","mediante","mejor","mencionó","menos","mi","misma","mismas","mismo","mismos","momento","mucha","muchas","mucho","nada","nadie","ni","ningún","ninguna","ningunas","ninguno","ningunos","no","nosotras","nuestra","nuestras","nuestro","nuestros","nueva","nuevas","nuevo","nuevos","nunca","o","ocho","otra","otras","otros","parece","parte","partir","pasada","pasado","pesar","poca","pocas","poco","pocos","podrá","podrán","podría","podrían","poner","posible","próximo","próximos","primer","primera","primeros","principalmente","propia","propias","propio","propios","pudo","pueda","pues","qué","que","quedó","queremos","quién","quienes","quiere","realizó","realizado","realizar","respecto","sí","sólo","se","señaló","sea","sean","según","segunda","segundo","seis","será","serán","sería","sido","siempre","siete","sigue","siguiente","sino","sola","solas","solos","son","tal","tampoco","tan","tanto","tenía","tendrá","tendrán","tenga","tenido","tercera","toda","todas","todavía","todos","total","trata","través","tres","tuvo","usted","varias","varios","veces","ver","vez","y","ya"]

In [20]:
#Palabras que son comunes en las consultas de países
stopwords+=["funda","ciudad","provincia","presidente"]

In [21]:
def eliminar_stopwords(texto, stopwords):
    return ' '.join([word for word in texto.split(' ') if word not in stopwords])

In [22]:
for i in range(len(df)):
  minusculas = df.iloc[i]['description'].lower()
  new_description = eliminar_stopwords(minusculas, stopwords)
  db.eventos_stopwords.update_many({'_id' : df.iloc[i]['_id']}, {'$set' : {"description" : new_description}}) 

## **Análisis Descriptivo**

### Funciones auxiliares

In [23]:
df['año'] = df['date'].str.split("/", n = 2, expand = True)[0]
df['mes'] = df['date'].str.split("/", n = 2, expand = True)[1]
df['dia'] = df['date'].str.split("/", n = 2, expand = True)[2]
df = df.sort_values(['año'])

In [24]:
def palabras_comunes(pais):
  titulo = "Palabras comunes en la historia de " + pais
  pais = pais.lower()
  regx = re.compile(pais, re.IGNORECASE)
  df_pais = pd.DataFrame(db.eventos.find({'description': regx}))

  # Función MAP
  mapper = bson.Code(
  '''
  function map(){
    var words = this.description.split(" ");
    
    if (words == null){
      return;
    }

    for (var i=0; i < words.length; i++){
      if(!words[i].includes("'''+pais+'''")){
        emit(words[i], 1);
      }
    }
  }
  ''')

  # Función REDUCE

  reducer = bson.Code(
  """
  function reduce(key, values) {
    return Array.sum(values)
  }    
  """)

  map_reduce_result = db.eventos_stopwords.map_reduce(mapper, 
                                          reducer, 
                                          'thing_count',
                                          query={'description': regx})
  
  top_words = pd.DataFrame(map_reduce_result.find()).sort_values(by='value', ascending=False).head(20)
  return px.bar(top_words, x = '_id', y = 'value', height=400, width=1000, title=titulo, 
       labels={
            "_id": "Palabras",
            "value": "Conteo"
        })


In [25]:
def eventos_pais(pais):
  titulo = "Eventos históricos de " + pais
  pais = pais.lower()
  regx = re.compile(pais, re.IGNORECASE)
  df_pais = pd.DataFrame(db.eventos.find({'description': regx}))
  df_pais['año'] = df_pais['date'].str.split("/", n = 2, expand = True)[0]
  df_pais['mes'] = df_pais['date'].str.split("/", n = 2, expand = True)[1]
  #df_pais = df_pais.astype({'año': 'int32', 'mes': 'int32'})
  df_pais = df_pais.sort_values(['año'])

  fig = px.scatter(df_pais,
        x='año',
        y='mes',
        labels = {'año','mes'},
        hover_data=['mes','description'],
        height=500, width=1000, title=titulo) 
  
  fig.update_yaxes(categoryorder="category ascending")
  #fig.update_xaxes(categoryorder="total ascending")
  return fig

In [26]:
def eventos_pais_lista(pais):
  return df[df.description.str.contains(pais, case=False)][['date','description', 'año', 'mes', 'dia']].sort_values(['año','mes', 'dia'])

In [27]:
datos_paises = pd.read_csv('paises.csv')
lista_paises_con_eventos = []

for pais in datos_paises['nombre']:
  conteo = df[df.description.str.contains(pais, case=False)]['_id'].count()
  if conteo > 0:
    lista_paises_con_eventos.append(True)
  else:
    lista_paises_con_eventos.append(False)

lista_paises_presentes = datos_paises[lista_paises_con_eventos]['nombre']

/usr/local/lib/python3.7/dist-packages/pandas/core/strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


In [28]:
def analisis_palabras(decisión, numero_palabras):

  if decisión == 'Todas':
    mapper = bson.Code(
    '''
    function map(){
      var words = this.description.split(" ");
      
      if (words == null){
        return;
      }

      for (var i=0; i < words.length; i++){
        emit(words[i], 1);
      }
    }
    ''')
  else:
    mapper = bson.Code(
    '''
    function map(){
      var words = this.description.split(" ");
      var seleted_words = ['guerra', 'atentados', 'ataque', 'muertos', 'descubrimiento', 'invención'];
      
      if (words == null){
        return;
      }

      for (var i=0; i < words.length; i++){
          if(seleted_words.includes(words[i])){
            emit(words[i], 1);
          }
      }
    }
    ''')

  # Función REDUCE
  reducer = bson.Code(
  """
  function reduce(key, values) {
    return Array.sum(values)
  }    
  """)

  map_reduce_result = db.eventos_stopwords.map_reduce(mapper, 
                                          reducer, 
                                          'thing_count')

  top_words = pd.DataFrame(map_reduce_result.find()).sort_values(by='value', ascending=False).head(numero_palabras)
  return px.bar(top_words, x = '_id', y = 'value', height=400, width=1000, 
                title="Análisis de palabras", 
                labels={
                      "_id": "Palabras",
                      "value": "Conteo"
                  })

In [29]:
df_años = df.astype({'año': 'int32'})
df_años = df_años.sort_values(['año'])
lista_años = df_años['año'].unique()

In [30]:
def eventos_por_año(año):
  año = str(año)
  pd.set_option('display.max_colwidth', None)
  pd.set_option('display.max_rows', None)
  new_df = df[df['año']==año][['date','description', 'mes', 'dia']].sort_values(['mes', 'dia'])
  return new_df

### Análisis

#### Análisis de palabras comunes en los eventos historicos de un país en concreto

In [31]:
display(Markdown('Seleccione el país:'))
interact(palabras_comunes, pais=lista_paises_presentes);

Seleccione el país:

interactive(children=(Dropdown(description='pais', options=('Afganistán', 'Albania', 'Alemania', 'Andorra', 'A…

#### Análisis de los eventos historicos ocurridos en un país en concreto

In [32]:
interact(eventos_pais, pais=lista_paises_presentes);

interactive(children=(Dropdown(description='pais', options=('Afganistán', 'Albania', 'Alemania', 'Andorra', 'A…

En forma de lista

In [33]:
interact(eventos_pais_lista, pais=lista_paises_presentes);

interactive(children=(Dropdown(description='pais', options=('Afganistán', 'Albania', 'Alemania', 'Andorra', 'A…

#### Análisis de palabras en el dataset

Seleccione en decisión:
*   Todas - para un análisis de todas las palabras presentes
*   Seleccionadas - para un análisis de las palabras seleccionadas previamente (guerra, atentados, ataque, muertos, descubrimiento, invención)

Seleccione en numero_palabras: el número de palabras que quiere ver en el gráfico

In [34]:
display(Markdown('### Análisis de palabras en el dataset'))
interact(analisis_palabras, decisión=['Todas', 'Seleccionadas'], numero_palabras = [i for i in range(10,61,1)]);

### Análisis de palabras en el dataset

interactive(children=(Dropdown(description='decisión', options=('Todas', 'Seleccionadas'), value='Todas'), Dro…

#### Análisis de eventos ocurridos por año

In [35]:
new_df = pd.DataFrame(df.groupby('año')['_id'].count())
new_df.index = new_df.index.map(int)
new_df = new_df.sort_index(ascending=True)
px.bar(new_df, y = '_id', 
       height=400, width=1000, title="Número de eventos por año", 
       labels={
            "_id": "Conteo"
        })

In [36]:
display(Markdown('Seleccione el año para ver los eventos ocurridos'))
interact(eventos_por_año, año=lista_años);

Seleccione el año para ver los eventos ocurridos

interactive(children=(Dropdown(description='año', options=(-299, -298, -297, -296, -295, -294, -293, -292, -29…